In [87]:
FEATURE_PER_DAY = 18
WINDOW_SIZE = 9
FEATURE_NUM = 18 * 9

In [88]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np

df = pd.read_csv("competitions/ml-2018spring-hw1/train.csv", encoding="big5")

# transform RAINFALL column to number
for row in np.arange(10, len(df), FEATURE_PER_DAY):
    df.iloc[row, 3:] = pd.to_numeric(df.iloc[row, 3:], errors="coerce")

df.fillna(0, inplace=True)

In [89]:
x_data = []
y_data = []
ROW_SIZE, COLUMN_SIZE = df.shape

for i in np.arange(0, len(df), FEATURE_PER_DAY):
    j = 3
    while float(df.iloc[i+9, j+WINDOW_SIZE]) > 0 and j + WINDOW_SIZE < COLUMN_SIZE - 1:
        arr = np.array(df.iloc[i:i+FEATURE_PER_DAY, j:j+WINDOW_SIZE], dtype=float)
        arr = np.insert(arr, 0, 1) # x0 for bias
        x_data.append(arr)
        y_data.append(np.array(df.iloc[i+9, j+WINDOW_SIZE], dtype=float))
        j += 1

In [90]:
# Initial model
theta = np.array([0.0] * (FEATURE_NUM + 1)) # all parameters init to 1
lr = 1
iteration = 10000

lr_ada = np.zeros(FEATURE_NUM + 1)

In [91]:
# Load initial parameters from npy file
# theta = np.load('theta.npy')

In [92]:
# Find the best function
from tqdm import tqdm

for i in tqdm(range(iteration)):
    grad = np.zeros(FEATURE_NUM + 1)
    
    for n in range(len(x_data)):
        dot = np.dot(theta, x_data[n])
        for f in range(len(grad)):
            grad[f] = grad[f] - 2.0 * (y_data[n] - dot) * x_data[n][f]
    
    for f in range(len(grad)):
        lr_ada[f] = lr_ada[f] + grad[f] ** 2

    for index in range(len(theta)):
        theta[index] = theta[index] - lr / np.sqrt(lr_ada[index]) * grad[index]

100%|██████████| 10000/10000 [2:08:07<00:00,  1.30it/s] 


In [94]:
# Verify model by compute its error
def computeCost(x_data, y_data, theta):
    cost = 0.0
    for i in range(len(x_data)):
        cost += (np.dot(theta, x_data[i]) - y_data[i]) ** 2
    return cost

In [98]:
# Save model to npy file
np.save('theta', theta)

In [103]:
# Compute test value
dt = pd.read_csv("competitions/ml-2018spring-hw1/test.csv", header=None, encoding="big5")

for row in np.arange(10, len(dt), FEATURE_PER_DAY):
    dt.iloc[row, 2:] = pd.to_numeric(dt.iloc[row, 2:], errors="coerce")
    
dt.fillna(0, inplace=True)

x_test = []
y_test = []

for i in np.arange(0, len(dt), FEATURE_PER_DAY):
    arr = np.array(dt.iloc[i:i+FEATURE_PER_DAY, 2:2+WINDOW_SIZE], dtype=float)
    arr = np.insert(arr, 0, 1) # x0 for bias
    x_test.append(arr)
    
for i in range(len(x_test)):
    y_test.append(np.dot(theta, x_test[i]))

# Replace negative value
for i in range(len(y_test)):
    if (y_test[i] < 0):
        y_test[i] = 0

arr = [["id_" + str(i), y_test[i]] for i in range(len(y_test))]
dw = pd.DataFrame(arr, columns = ["id", "value"])
dw.to_csv("output.csv", index=False)